# 크롤링 과제
url => https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page=1

과제: 무신사 코디맵 페이지(많은 댓글 수 기준 정렬)에 나와있는 60개의 코디맵에 대하여 각각 ['codimap_category', 'codimap_title', 'codimap_date', 'views', 'comment_numbers'] 정보를 가져와서 csv 형태로 만든 다음, 해당 코드(.ipynb)와 csv 파일을 본인 github 과제 repository에 제출

Challenge:

각 코디맵마다 클릭한 뒤 다시 이전페이지로 돌아오는 코드를 반복문에 추가하여 ['codimap_explain', 'codimap_hashtag', 'codimap_imgurl'] 정보도 가져오기
page url을 format 함수를 이용해 수정한 뒤 반복문을 통해 1페이지부터 5페이지까지의 코디맵 정보 가져오기
Challenge는 Optional Task로 필수적으로 해야하는 것은 아닙니다. 좀 더 심화된 크롤링을 원할경우 시도해보면 좋을 것 같아요!

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium.webdriver.common.by import By

In [2]:
# ChromeDriver 초기화
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 크롤링할 페이지 URL
url = 'https://www.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=comment_cnt&page=1'

# 페이지 열기
driver.get(url)

# BeautifulSoup 객체 생성
soup = BeautifulSoup(driver.page_source, 'html.parser')

# 데이터를 저장할 리스트 초기화
codimap_list = []
codicont_list = []

In [3]:
# 동적 페이지 스크롤
for codimap in range(2):
    interval = 1 # 1초에 한번씩 스크롤 내림

    # 현재 문서 높이를 가져와서 저장
    prev_height = driver.execute_script("return document.body.scrollHeight")

    # 반복 수행
    while True:
        # 스크롤을 가장 아래로 내림
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

        # 페이지 로딩 대기
        time.sleep(interval)

        # 현재 문서 높이를 가져와서 저장
        curr_height = driver.execute_script("return document.body.scrollHeight")
        if curr_height == prev_height:
            break

        prev_height = curr_height

In [4]:
 # BeautifulSoup 객체 생성
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [5]:
# li 요소를 찾아서 반복문으로 데이터 추출
for li in soup.find_all('li', class_='style-list-item'):
    # 데이터 추출
    category = li.find('span', class_='style-list-information__text').text.strip()
    title = li.find('strong', class_='style-list-information__title').text.strip()
    date = li.find_all('span', class_='post-information__text')[0].text.strip()
    view = li.find_all('span', class_='post-information__text')[1].text.strip()
    comment = li.find_all('span', class_='post-information__text')[2].text.strip()
    
    # 추출한 데이터를 리스트에 추가
    codimap_list.append([category, title, date, view, comment])

In [17]:
# 데이터프레임 생성
df = pd.DataFrame(codimap_list, columns=['category', 'title', 'date', 'view', 'comment'])

In [18]:
from datetime import datetime

def convert_date(date_string):
    date_format = '%y.%m.%d'  # 주어진 문자열의 형식에 맞게 포맷을 설정합니다.
    date_object = datetime.strptime(date_string, date_format).date()
    formatted_date = date_object.strftime('%Y-%m-%d')  # 원하는 날짜 형식으로 포맷을 변경합니다.
    return formatted_date

# 데이터프레임의 'date' 컬럼에 있는 값을 변환
df['date'] = df['date'].apply(convert_date)

In [19]:
# view 컬럼의 '조회 ', ',제거
df['view'] = df['view'].str.replace('조회 ', '')
df['view'] = df['view'].str.replace(',', '')

# comment 컬럼의 '댓글' 제거
df['comment'] = df['comment'].str.replace('댓글 ', '')

# category와 title 컬럼의 앞 뒤 공백 제거
df['category']  = df['category'].str.strip()
df['title']  = df['title'].str.strip()

In [21]:
df.comment = df.comment.apply('int64')
df.view = df.view.apply('int64')

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  60 non-null     object
 1   title     60 non-null     object
 2   date      60 non-null     object
 3   view      60 non-null     int64 
 4   comment   60 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 2.5+ KB


In [23]:
df

,category,title,date,view,comment
0,캐주얼,톤온톤 코디,2021-07-02,15982,49
1,캐주얼,블랙 매니아,2021-02-23,34648,46
2,캐주얼,오늘의 코디,2022-03-22,12177,38
3,댄디,추천 댄디 룩,2021-09-15,20166,27
4,캐주얼,릴랙스 룩,2021-02-23,23518,22
5,스트릿,힙하다 힙해,2021-02-03,35269,21
6,캐주얼,심플한 코디,2021-07-30,39553,18
7,댄디,데이트 코디,2021-12-24,10294,18
8,댄디,훈훈 남친 룩,2021-03-24,19327,16
9,캐주얼,지금 입기 좋아,2021-08-31,20198,16


In [24]:
df.to_csv('musinsa_crawling.csv', index=False, encoding='utf-8-sig')

In [25]:
# 크롬 드라이버 종료
driver.quit()